In [ ]:
import sagemaker

print (sagemaker.__version__)

In [ ]:
dataset_uri = 

In [ ]:
from sagemaker.automl.automl import AutoML

job = AutoML(
    role=sagemaker.get_execution_role(),
    problem_type='BinaryClassification',      # Optional
    job_objective={'MetricName': 'AUC'},      # Optional
    target_attribute_name='credit__default',
    max_candidates=250 # Optional
)

In [ ]:
job.fit(inputs=dataset_uri, wait=False)

In [ ]:
from time import sleep

def wait_for(job, status):
    response = job.describe_auto_ml_job()
    job_status = response['AutoMLJobStatus']
    job_sec_status = response['AutoMLJobSecondaryStatus']

    if job_status not in ('Stopped', 'Failed'):
        while job_status in ('InProgress') and job_sec_status in (status):
            response = job.describe_auto_ml_job()
            job_status = response['AutoMLJobStatus']
            job_sec_status = response['AutoMLJobSecondaryStatus']
            print (job_status, job_sec_status)
            sleep(60)

In [ ]:
wait_for(job, 'AnalyzingData')

In [ ]:
wait_for(job, 'FeatureEngineering')

In [ ]:
wait_for(job, 'ModelTuning')

In [ ]:
job.describe_auto_ml_job()

In [ ]:
job.best_candidate()

In [ ]:

job.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    endpoint_name=endpoint_name
)

In [ ]:
import boto3

endpoint_name = 'automl-credit-decisions'

sm = boto3.client('sagemaker')
sm_rt = boto3.client('sagemaker-runtime')

In [ ]:
payload = '0,0,0,foreign,labourer,no_account,2,2,1,very_high,2,250,False,3,6,very_poor,real_estate,new_car,own,2'

In [ ]:
response = sm_rt.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType='text/csv'
)

response['Body'].read().decode('utf-8')

In [ ]:
#sm.delete_endpoint(endpoint_name=endpoint_name)